# Locating Multivalued and Duplicate records (single csv file)
### Saksham Gakhar, DA - DKSF

Keep changing the input csv file and look for duplicate and multivalued records, enlist devices that generally misbehave...

In [45]:
import numpy as np 
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from collections import defaultdict
from datetime import datetime
# without mpld3
%matplotlib notebook 

# with mpld3
# %matplotlib inline
# import mpld3
# mpld3.enable_notebook()

In [33]:
df = pd.read_csv('2020-07-06-DataKind/output-2020-04-01T00_00_00+00_00.csv')

In [34]:
df.when_captured = pd.to_datetime(df.when_captured)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56799 entries, 0 to 56798
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   service_uploaded  56799 non-null  object             
 1   when_captured     56799 non-null  datetime64[ns, UTC]
 2   device_urn        56799 non-null  object             
 3   device_sn         56799 non-null  object             
 4   device            56799 non-null  int64              
 5   loc_lat           56799 non-null  float64            
 6   loc_lon           56799 non-null  float64            
 7   env_temp          16830 non-null  float64            
 8   env_humid         16830 non-null  float64            
 9   pms_pm01_0        23026 non-null  float64            
 10  pms_pm02_5        23030 non-null  float64            
 11  pms_pm10_0        23026 non-null  float64            
 12  lnd_7318c         33293 non-null  float64            
 13  l

Need to change the format of the Time Stamp for all the measurements in the raw data

In [35]:
# df.service_uploaded =  df.service_uploaded.apply(lambda x: datetime.strptime(x, '%b %d, %Y @ %H:%M:%S.%f'))

In [36]:
df[0:5]

,service_uploaded,when_captured,device_urn,device_sn,device,loc_lat,loc_lon,env_temp,env_humid,pms_pm01_0,pms_pm02_5,pms_pm10_0,lnd_7318c,lnd_7318u,bat_voltage
0,"Apr 30, 2020 @ 23:59:23.000",2020-04-30 23:59:23+00:00,safecast:3714913954,Solarcast #30027,3714913954,52.395,4.875,NaN,NaN,NaN,NaN,NaN,23.0,25,NaN
1,"Apr 30, 2020 @ 23:57:54.000",2020-04-30 23:57:54+00:00,safecast:678194983,Solarcast #30030,678194983,46.554,15.635,NaN,NaN,NaN,NaN,NaN,24.0,29,3.876
2,"Apr 30, 2020 @ 23:57:37.000",2020-04-30 23:57:39+00:00,safecast:678194983,Solarcast #30030,678194983,46.554,15.635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Apr 30, 2020 @ 23:57:20.000",2020-04-30 23:57:15+00:00,safecast:678194983,Solarcast #30030,678194983,46.554,15.635,NaN,NaN,1.0,2.0,2.0,NaN,NaN,NaN
4,"Apr 30, 2020 @ 23:45:01.000",2020-04-30 23:45:01+00:00,safecast:3714913954,Solarcast #30027,3714913954,52.395,4.875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56799 entries, 0 to 56798
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   service_uploaded  56799 non-null  object             
 1   when_captured     56799 non-null  datetime64[ns, UTC]
 2   device_urn        56799 non-null  object             
 3   device_sn         56799 non-null  object             
 4   device            56799 non-null  int64              
 5   loc_lat           56799 non-null  float64            
 6   loc_lon           56799 non-null  float64            
 7   env_temp          16830 non-null  float64            
 8   env_humid         16830 non-null  float64            
 9   pms_pm01_0        23026 non-null  float64            
 10  pms_pm02_5        23030 non-null  float64            
 11  pms_pm10_0        23026 non-null  float64            
 12  lnd_7318c         33293 non-null  float64            
 13  l

In [38]:
temp_df = df.groupby(['device_urn', 'device_sn','when_captured']).size().to_frame('size').reset_index().sort_values('size', ascending=False)

In [39]:
temp_df[(temp_df['size']>1)]

,device_urn,device_sn,when_captured,size
15012,safecast:2152053642,Solarcast #30025,2020-04-07 15:59:56+00:00,24
1678,safecast:1094924990,Solarcast #30024,2020-04-15 22:36:25+00:00,19
32529,safecast:3768313999,Solarcast #30026,2020-04-20 05:44:22+00:00,19
15341,safecast:2152053642,Solarcast #30025,2020-04-20 03:24:01+00:00,17
32057,safecast:3768313999,Solarcast #30026,2020-04-06 06:24:45+00:00,17
...,...,...,...,...
31863,safecast:3714913954,Solarcast #30027,2020-04-30 16:14:23+00:00,2
31166,safecast:3714913954,Solarcast #30027,2020-04-27 08:44:24+00:00,2
28394,safecast:3714913954,Solarcast #30027,2020-04-13 18:22:34+00:00,2
28269,safecast:3714913954,Solarcast #30027,2020-04-13 03:52:46+00:00,2


Devices that have misbehaved at some point - more than one data values per time stamp


In [40]:
np.unique(temp_df[temp_df['size']>1]['device_sn'].values) # devices that have misbehaved

array(['Solarcast #30000', 'Solarcast #30001', 'Solarcast #30002',
       'Solarcast #30003', 'Solarcast #30006', 'Solarcast #30007',
       'Solarcast #30012', 'Solarcast #30013', 'Solarcast #30014',
       'Solarcast #30016', 'Solarcast #30017', 'Solarcast #30018',
       'Solarcast #30019', 'Solarcast #30022', 'Solarcast #30023',
       'Solarcast #30024', 'Solarcast #30025', 'Solarcast #30026',
       'Solarcast #30027', 'Solarcast #33001', 'Solarcast #33003',
       'Solarcast #33008', 'Solarcast #33009'], dtype=object)

Based on above table for (`device`, `when_captured`) key, let's see what these multiple values for each time stamp correspond to. Sometimes there are negative RH, sometimes 0.0 PM (which measn very clean air)

In [44]:
df[df['when_captured']=='2020-04-07 15:59:56+00:00']

,service_uploaded,when_captured,device_urn,device_sn,device,loc_lat,loc_lon,env_temp,env_humid,pms_pm01_0,pms_pm02_5,pms_pm10_0,lnd_7318c,lnd_7318u,bat_voltage,year
39456,"Apr 7, 2020 @ 15:59:56.000",2020-04-07 15:59:56+00:00,safecast:2152053642,Solarcast #30025,2152053642,34.122,-118.261,NaN,NaN,NaN,NaN,NaN,29.0,36,NaN,2020
39457,"Apr 7, 2020 @ 15:59:56.000",2020-04-07 15:59:56+00:00,safecast:2152053642,Solarcast #30025,2152053642,34.122,-118.261,NaN,NaN,NaN,NaN,NaN,31.0,30,NaN,2020
39458,"Apr 7, 2020 @ 15:59:56.000",2020-04-07 15:59:56+00:00,safecast:2152053642,Solarcast #30025,2152053642,34.122,-118.261,NaN,NaN,NaN,NaN,NaN,37.0,41,NaN,2020
39459,"Apr 7, 2020 @ 15:59:56.000",2020-04-07 15:59:56+00:00,safecast:2152053642,Solarcast #30025,2152053642,34.122,-118.261,10.380,90.348,1.0,2.0,2.0,34.0,37,2.722,2020
39460,"Apr 7, 2020 @ 15:59:56.000",2020-04-07 15:59:56+00:00,safecast:2152053642,Solarcast #30025,2152053642,34.122,-118.261,8.023,87.093,NaN,NaN,NaN,32.0,27,2.663,2020
39461,"Apr 7, 2020 @ 15:59:56.000",2020-04-07 15:59:56+00:00,safecast:2152053642,Solarcast #30025,2152053642,34.122,-118.261,10.553,91.799,1.0,2.0,3.0,35.0,36,2.647,2020
39462,"Apr 7, 2020 @ 15:59:56.000",2020-04-07 15:59:56+00:00,safecast:2152053642,Solarcast #30025,2152053642,34.122,-118.261,8.751,81.998,NaN,NaN,NaN,0.0,0,2.607,2020
39463,"Apr 7, 2020 @ 15:59:56.000",2020-04-07 15:59:56+00:00,safecast:2152053642,Solarcast #30025,2152053642,34.122,-118.261,8.466,88.449,NaN,NaN,NaN,37.0,34,2.714,2020
39464,"Apr 7, 2020 @ 15:59:56.000",2020-04-07 15:59:56+00:00,safecast:2152053642,Solarcast #30025,2152053642,34.122,-118.261,9.458,89.718,0.0,1.0,1.0,35.0,37,2.843,2020
39465,"Apr 7, 2020 @ 15:59:56.000",2020-04-07 15:59:56+00:00,safecast:2152053642,Solarcast #30025,2152053642,34.122,-118.261,8.532,82.922,NaN,NaN,NaN,0.0,0,2.676,2020


Add a column for the year

In [42]:
df['year'] = pd.DatetimeIndex(df['when_captured']).year

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56799 entries, 0 to 56798
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   service_uploaded  56799 non-null  object             
 1   when_captured     56799 non-null  datetime64[ns, UTC]
 2   device_urn        56799 non-null  object             
 3   device_sn         56799 non-null  object             
 4   device            56799 non-null  int64              
 5   loc_lat           56799 non-null  float64            
 6   loc_lon           56799 non-null  float64            
 7   env_temp          16830 non-null  float64            
 8   env_humid         16830 non-null  float64            
 9   pms_pm01_0        23026 non-null  float64            
 10  pms_pm02_5        23030 non-null  float64            
 11  pms_pm10_0        23026 non-null  float64            
 12  lnd_7318c         33293 non-null  float64            
 13  l